In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-28 00:04:57--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Connecting to 127.0.0.1:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0.05s   

2024-06-28 00:04:57 (68.8 KB/s) - ‘minsearch.py’ saved [3832/3832]



In [5]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-28 00:06:02--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Connecting to 127.0.0.1:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  1.26MB/s    in 0.5s    

2024-06-28 00:06:03 (1.26 MB/s) - ‘documents.json’ saved [658332/658332]



In [2]:
import minsearch

In [6]:
import json

In [7]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [8]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [11]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [22]:
q = 'the course has already started, can I still enroll?'

In [23]:
index.fit(documents)

In [32]:
from openai import OpenAI
from groq import Groq

from dotenv import load_dotenv

load_dotenv()

True

In [33]:
# client = OpenAI()
client = Groq()

In [34]:
response = client.chat.completions.create(
    # model='gpt-4o',
    model="llama3-8b-8192",
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

'Unfortunately, the answer to this question is often "no".\n\nTypically, enrolling into a course requires you to register and pay fees before the course starts. Once the course has already begun, it\'s usually not possible to enroll.\n\nSome exceptions might include:\n\n1. If the course allows late registration or has an open enrollment period, you might still be able to sign up.\n2. If you\'re transferring from another institution or taking a course as a continuing education professional development opportunity, some courses might be opened up for late entry.\n3. In rare cases, some online courses or distance learning programs might allow students to start at any time, so you could still enroll and begin the course from where others have left off.\n\nTo find out if you can still enroll, I\'d suggest contacting the institution, professor, or course administrator directly and asking:\n\n* Is the course still accepting new students?\n* Are there any late registration fees or deadlines I 

In [38]:
print(response.choices[0].message.content)

Unfortunately, the answer to this question is often "no".

Typically, enrolling into a course requires you to register and pay fees before the course starts. Once the course has already begun, it's usually not possible to enroll.

Some exceptions might include:

1. If the course allows late registration or has an open enrollment period, you might still be able to sign up.
2. If you're transferring from another institution or taking a course as a continuing education professional development opportunity, some courses might be opened up for late entry.
3. In rare cases, some online courses or distance learning programs might allow students to start at any time, so you could still enroll and begin the course from where others have left off.

To find out if you can still enroll, I'd suggest contacting the institution, professor, or course administrator directly and asking:

* Is the course still accepting new students?
* Are there any late registration fees or deadlines I need to be aware 

In [39]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [40]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [41]:
def llm(prompt):
    response = client.chat.completions.create(
        # model='gpt-4o',
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [42]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [43]:
rag(query)

"Based on the context, I assume the question is asking how to run Kafka.\n\nI can't find a direct answer to running Kafka in the provided context. The context seems to be about setting up and running applications that use Kafka, such as producers, consumers, KStreams, etc. However, I can provide some general information.\n\nTo run Kafka, you typically need to start a Kafka cluster with multiple nodes (i.e., brokers). You can do this by running a command like this:\n\n>kafka-server-start.sh config/server.properties\n\nThis command starts a Kafka broker with the configuration specified in the `server.properties` file.\n\nHowever, please note that the context is primarily focused on running Java and Python applications that use Kafka, rather than running Kafka itself."

In [45]:
print(rag('the course has already started, can I still enroll?'))

Based on the provided context, I would answer the QUESTION: "The course has already started, can I still enroll?"

Answer: Yes, you can still enroll in the course, even though it has already started. The FAQ mentions that even if you don't register initially, you're still eligible to submit the homeworks. However, please be aware of the deadlines for turning in the final projects, so don't leave everything for the last minute.


In [46]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [20]:
from elasticsearch import Elasticsearch

In [21]:
es_client = Elasticsearch('http://localhost:9200') 

In [23]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████| 948/948 [00:28<00:00, 33.07it/s]


In [36]:
query = 'I just disovered the course. Can I still join it?'

In [42]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [44]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [45]:
rag(query)

'Yes, you can still join the course even if you discovered it after the start date. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects. So make sure not to leave everything for the last minute.'